# Drone Synth PoC

1. Generate notes in A minor 7th chord
1. Create 20 sawtooth waveform oscillators per note, assigning random detune to each around the base frequency for a total of 80 oscillators
1. IIR reverb
1. Low pass filter
1. Sub bass note one active below A2 with harmonics
1. Soft distortion to sub bass via tanh
1. Tremolo via LFO to create movement
1. Apply fade out envelope once notes stop playing
1. Normalize output

In [ ]:
import numpy as np
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from scipy.signal import convolve
from scipy import signal
import soundfile as sf
from scipy.io.wavfile import write

def reverb(waveform, reverb_file="../../resources/hum_ir.wav", gain_dry=1.0, gain_wet=1.0, output_gain=0.05):
    ir, fs_ir = sf.read(reverb_file)
    if ir.ndim == 1:
        ir = np.array([ir, ir])
    elif ir.shape[1] == 2:
        ir = ir.T
    else:
        ir = ir[:, :2].T
    
    # Normalize each IR channel
    ir[0] /= np.max(np.abs(ir[0]))
    ir[1] /= np.max(np.abs(ir[1]))
    
    if waveform.ndim == 1:
        waveform = np.array([waveform, waveform])
    elif waveform.shape[0] != 2:
        raise ValueError("Input waveform must be mono or stereo")

    output_len = waveform.shape[1] + ir.shape[1] - 1
    output = np.zeros([2, output_len], dtype=np.float64)
    
    output[0] = output_gain * convolve(waveform[0] * gain_dry, ir[0] * gain_wet, method='fft')
    output[1] = output_gain * convolve(waveform[1] * gain_dry, ir[1] * gain_wet, method='fft')
    
    return output


def generate_chord(base_freq, chord_type="major", include_seventh=True, seventh_type="maj7"):
    if chord_type == "major":
        third = 4
    elif chord_type == "minor":
        third = 3
    else:
        raise ValueError("chord_type must be 'major' or 'minor'")
    
    fifth = 7
    
    notes = [
        base_freq,
        base_freq * 2**(third/12),
        base_freq * 2**(fifth/12)
    ]
    
    if include_seventh:
        if seventh_type == "maj7":
            seventh = 11
        elif seventh_type == "min7":
            seventh = 10
        else:
            raise ValueError("seventh_type must be 'maj7' or 'min7'")
        notes.append(base_freq * 2**(seventh/12))
    
    return notes

def multi_voice_oscillator(freq, t, amp=1.0, detune_range=2, voices=5, wave_type="sine"):
    total = np.zeros_like(t)
    for i in range(voices):
        detune = np.random.uniform(-detune_range, detune_range)
        f = freq + detune

        if wave_type == "sine":
            osc = np.sin(2 * np.pi * f * t)
        elif wave_type == "saw":
            phase = (f * t) % 1.0
            osc = 2 * phase - 1
        elif wave_type == "triangle":
            phase = (f * t) % 1.0
            osc = 2 * np.abs(2 * phase - 1) - 1
        else:
            raise ValueError("Unknown wave_type")

        total += amp * osc

    return total

def oscillator(freq, t, amp=1.0, detune=0.0):
    return amp * np.sin(2 * np.pi * (freq + detune) * t)




Settings/Initial waveform

In [ ]:
# Settings
fs = 44100
duration = 10
t = np.linspace(0, duration, int(fs*duration), endpoint=False)

# Base frequencies for a nice drone chord
base_freq = 110 # A2
chord = generate_chord(base_freq, chord_type="minor", include_seventh=True, seventh_type="min7")

# Mix oscillators with slight detune and modulation
drone = sum(multi_voice_oscillator(f, t, amp=0.5, detune_range=2, voices=8, wave_type="saw") for f in chord)

Drone Rendering

In [ ]:
# Reverb
drone = reverb(drone, reverb_file="../../resources/hum_ir.wav")
#drone = reverb(drone, reverb_file="../../resources/darker_ir.wav")
drone = reverb(drone, reverb_file="../../resources/dark_ir.wav", gain_dry=0.8, gain_wet=0.5, output_gain=0.1)

# Low-pass filter parameters
cutoff_freq = 300
order = 5
sos = signal.iirfilter(order, cutoff_freq, btype='lowpass', ftype='butter', fs=fs, output='sos')
drone = signal.sosfilt(sos, drone)

# Sub bass
sub_freq = base_freq / 2
sub_bass = np.sin(2 * np.pi * sub_freq * t) * 0.6

# add some gentle harmonics
sub_bass += 0.2 * np.sin(2 * np.pi * 2 * sub_freq * t)
sub_bass += 0.1 * np.sin(2 * np.pi * 3 * sub_freq * t)

# optional: soft saturation for organic feel
sub_bass = np.tanh(sub_bass * 0.8)

# pad and add to drone
sub_bass = np.pad(sub_bass, (0, drone.shape[1] - len(sub_bass)))
drone[0, :] += sub_bass
drone[1, :] += sub_bass

# Tremolo effect
output_len = drone.shape[1]
tremolo_freq = 0.01
tremolo = 0.5 + 0.5 * np.sin(2 * np.pi * tremolo_freq * np.linspace(0, duration, output_len))

# Apply release to tremolo
release_time = 5.0
n_release = int(fs * release_time)
release_env = np.ones(output_len)
release_env[-n_release:] = np.linspace(1.0, 0.0, n_release)

# Combine tremolo with release
env = tremolo * release_env
drone *= env

# Normalize & Output
drone /= (np.max(np.abs(drone)) * 1.05)
display(Audio(drone, rate=fs))

# Pick one channel (stereo → mono for analysis)
mono_drone = drone[0, :]  
spectrum = np.fft.rfft(mono_drone)
freqs = np.fft.rfftfreq(len(mono_drone), 1/fs)

# Convert magnitude to dB
magnitude_db = 20 * np.log10(np.abs(spectrum) + 1e-6)

# Plot
plt.figure(figsize=(12,6))
plt.semilogx(freqs, magnitude_db)
plt.title("Drone")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude (dB)")
plt.xlim(20, 5000)
plt.grid(True, which="both")
plt.show()